### An Application: crime data

Here's a simple application using one of the UCI data sets: communities and crime.

Let's first load the data and do some basic investigations.

In [ ]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import matplotlib as mp
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# regularization methos
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [ ]:
# from: https://github.com/justmarkham/DAT5/blob/master/code/18_regularization.py

########## Prepare data ##########
# read in data, remove categorical features, remove rows with missing values

crime = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.data', 
                    header=None, na_values=['?'])
crime = crime.iloc[:, 5:] # get rid of columns that are the names of towns
crime.dropna(inplace=True) # drop missing values
print(crime.head())

# the names of the columns
crime_names = np.genfromtxt('comm_names.txt',dtype='str')[5:, :]
## Meaning of names: https://archive.ics.uci.edu/ml/datasets/Communities%2Band%2BCrime

# define X and y; 
# y from the last column which is the:
# ViolentCrimesPerPop: total number of violent crimes per 100K popuation (numeric - decimal) 
# GOAL attribute (to be predicted)
# (from: https://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.names)
X = crime.iloc[:, :-1]
y = crime.iloc[:, -1]

# split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# now, fit a few models, let's try linear regresion, ridge regression, and the lasso

# linear regression, ridge, lasso

lm = LinearRegression()
lm.fit(X_train, y_train)

lm_test_mse = ((lm.predict(X_test) - y_test) ** 2).mean()

# ridge regression
rm = Ridge(alpha=1, normalize=False)
rm.fit(X_train, y_train)

rm_test_mse = ((rm.predict(X_test) - y_test) ** 2).mean()

# lasso regression
las = Lasso(alpha=0.01, normalize=True, max_iter=100)
las.fit(X_train, y_train)

las_test_mse = ((las.predict(X_test) - y_test) ** 2).mean()

print("Linear model test MSE:     ", lm_test_mse)
print("Ridge regression test MSE: ", rm_test_mse)
print("Lasso regression test MSE: ", las_test_mse)

In [ ]:
# what about the coefficients; what does lasso drop?

# extract coefficients
lm_coef = lm.coef_
rm_coef = rm.coef_
las_coef = las.coef_

print("Non-zero lasso coefficients: \n", crime_names[np.where(las_coef!=0), :])

# plotting
fig = plt.figure()
plt.scatter(lm_coef, rm_coef)
fig.suptitle('Coefficients: lm vs ridge', fontsize=20)
plt.xlabel('LM coefficient')
plt.ylabel('Ridge coefficient')

fig = plt.figure()
plt.scatter(lm_coef, las_coef)
fig.suptitle('Coefficients: lm vs lasso', fontsize=20)
plt.xlabel('LM coefficient')
plt.ylabel('LASSO coefficient')

In [ ]:
# looking at the above plot we see that there are a couple of really big coefficients
# what are these?
print("Big lm coefficients: \n")
big_coefs = np.where(abs(lm_coef) > 5)
print(big_coefs, '\n')

print(lm_coef[big_coefs])
print(crime_names[big_coefs, :])
print('correlation between the big coefficients features: \n')
print(np.corrcoef(np.array(X_train)[:, big_coefs[0][0]],
                 np.array(X_train)[:, big_coefs[0][1]]))

How can we choose the best alpha? We'll use cross validation here.

In [ ]:
# investigate those two highly correlated features; what do lasso, ridge do to them?

print(big_coefs)
print()

alpha_vec = [0.000000001, 0.00000001, 0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, .1, 1, 10, 100]
coef_idx = 103
rm_coef_values = np.zeros(len(alpha_vec))
las_coef_values = np.zeros(len(alpha_vec))
lm_coef_line = np.zeros(len(alpha_vec))

for aa_idx, aa in enumerate(alpha_vec):
    lm = LinearRegression()
    lm.fit(X_train, y_train)

    rm = Ridge(alpha=aa, normalize=True)
    rm.fit(X_train, y_train)
    
    las = Lasso(alpha=aa, normalize=True, max_iter=100)
    las.fit(X_train, y_train)
    
    rm_coef = rm.coef_[coef_idx]
    las_coef = las.coef_[coef_idx]
    lm_coef = lm.coef_[coef_idx]
 
    rm_coef_values[aa_idx] = rm_coef
    las_coef_values[aa_idx] = las_coef
    lm_coef_line[aa_idx] = lm_coef

print('Ridge: coefficients', coef_idx, 'for alphas:\n', rm_coef_values.reshape((-1,1)))
print('Lasso: coefficients', coef_idx, 'for alphas:\n', las_coef_values.reshape((-1,1)) )

#plt.plot(np.log(alpha_vec), lm_coef_line * 0)
plt.plot(np.log(alpha_vec), rm_coef_values, label='Ridge')
plt.plot(np.log(alpha_vec), las_coef_values, label = 'Lasso')
plt.plot(np.log(alpha_vec), lm_coef_line, label='No reg')
plt.legend()
plt.title('Lasso; Ridge (g, b) regularization path\n for coefficient ' + str(coef_idx) + 
          '; lm value (red straight line): ' + str(lm_coef))
plt.ylabel('Coefficient Values') 
plt.xlabel('log(Alpha)') 
plt.show()

In [ ]:
# cross validation to select the best alpha parameters

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer

## IMPLEMENT: how can we use cross validation to choose the best parameters?

rm = Ridge(normalize=True)
rm_params = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1.0, 10., 100.]}
rm_grid = GridSearchCV(rm, rm_params, iid = True, cv=3)

rm_grid.fit(X_train, y_train)

best_rm_alpha = rm_grid.best_params_['alpha']
print('Best rm alpha:', best_rm_alpha)


las = Lasso(normalize=True, max_iter = 100)
las_params = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1.0, 10., 100.]}
las_grid = GridSearchCV(las, las_params, iid = True, cv=3)

las_grid.fit(X_train, y_train)

best_las_alpha = las_grid.best_params_['alpha']
print('Best las alpha:', best_las_alpha)

In [ ]:
# finally, re run all the models using the best alpha values as found 
# by cross validation and plot the results

lm = LinearRegression()
lm.fit(X_train, y_train)

lm_test_mse = ((lm.predict(X_test) - y_test) ** 2).mean()

rm_best = Ridge(normalize=False, alpha = best_rm_alpha)
rm_best.fit(X_train, y_train)
rm_test_mse = ((rm_best.predict(X_test) - y_test) ** 2).mean()

las_best = Lasso(normalize=True, max_iter=100, alpha = best_las_alpha)
las_best.fit(X_train, y_train)
las_test_mse = ((las_best.predict(X_test) - y_test) ** 2).mean()

print("Linear model test MSE:     ", lm_test_mse)
print("Ridge regression test MSE @ best CV alpha: ", rm_test_mse)
print("Lasso regression test MSE @ best CV alpha: ", las_test_mse)

print("Non-zero lasso coefficients: \n")

for i in range(len(crime_names[np.where(las_best.coef_!=0), :][0])):
    print(crime_names[np.where(las_best.coef_!=0), :][0][i], " : ", las_best.coef_[np.where(las_best.coef_!=0)[0][i]], " : ", rm_best.coef_[np.where(las_best.coef_!=0)[0][i]])



In [ ]:
print("\n\nDropped (zero) lasso coefficients: \n")

for i in range(len(crime_names[np.where(las_best.coef_==0), :][0])):
    print(crime_names[np.where(las_best.coef_==0), :][0][i], " : ", las_best.coef_[np.where(las_best.coef_==0)[0][i]], " : ", rm_best.coef_[np.where(las_best.coef_==0)[0][i]])
